In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Loading Data, Basic Check

### Load Data

In [17]:
df = pd.read_csv('Hospital_Inpatient_Discharges__SPARCS_De-Identified___Cost_Transparency__Beginning_2009_20250426.csv')

### Basic Check

In [18]:
df.head()

,Year,Facility Id,Facility Name,APR DRG Code,APR Severity of Illness Code,APR DRG Description,APR Severity of Illness Description,APR Medical Surgical Code,APR Medical Surgical Description,Discharges,Mean Charge,Median Charge,Mean Cost,Median Cost
0,2016,4,Albany Memorial Hospital,194,1,Heart Failure,Minor,M,Medical,2,"8,375.41","8,375.41","3,585.05","3,585.05"
1,2016,4,Albany Memorial Hospital,194,2,Heart Failure,Moderate,M,Medical,40,"14,029.82","12,176.95","6,182.67","5,253.15"
2,2016,4,Albany Memorial Hospital,194,3,Heart Failure,Major,M,Medical,70,"23,921.77","20,229.81","11,149.49","9,068.10"
3,2016,4,Albany Memorial Hospital,194,4,Heart Failure,Extreme,M,Medical,12,"51,260.45","35,210.82","26,081.70","15,230.62"
4,2016,4,Albany Memorial Hospital,196,4,Cardiac Arrest,Extreme,M,Medical,1,"25,357.84","25,357.84","7,791.75","7,791.75"


Based on the dataset documentation, the following pairs of columns are duplicative: 

* `Facility Id` and `Facility Name`
* `APR DRG Code` and `APR DRG Description`
* `APR Severity of Illness Code` and `APR Severity of Illness Description`
* `APR Medical Surgical Code` and `APR Medical Surgical Description`. 

To streamline the dataset and avoid redundancy, we decided to retain the coded columns and drop the corresponding descriptive columns.

In [ ]:
columns_to_drop = [
    'Facility Name', 
    'APR Severity of Illness Description', 
    'APR DRG Description', 
    'APR Medical Surgical Description',
]

df = df.drop(columns=columns_to_drop)

Based on observations from the Data Wrangler tool, the `APR Severity of Illness` column contains 351 missing values. As this represents less than 1% of the total observations, we have determined that it is appropriate to remove these records with missing values to maintain data quality without significantly impacting the dataset size.

We also removed rows where 'APR Medical Surgical Code' is labeled as 'U' which stands for others. (less than 1%)

In [22]:
df = df.dropna()
df = df[df['APR Medical Surgical Code'] != 'U']

Check the data type of each column

In [25]:
print(df.dtypes)

Year                             int64
Facility Id                      int64
APR DRG Code                     int64
APR Severity of Illness Code     int64
APR Medical Surgical Code       object
Discharges                      object
Mean Charge                     object
Median Charge                   object
Mean Cost                       object
Median Cost                     object
dtype: object


Columns `Discharges`, `Mean Charge`, `Median Charge`, `Mean Cost`, and `Median Cost` are expected to be numeric. But currently they are stored as object data types. To ensure appropriate data handling and enable quantitative analysis, we have decided to convert these columns to integer.

In [26]:
for col in ['Discharges', 'Mean Charge', 'Median Charge', 'Mean Cost', 'Median Cost']:
    df[col] = df[col].replace('[\\$,]', '', regex=True).astype(float)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1192267 entries, 0 to 1192826
Data columns (total 10 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   Year                          1192267 non-null  int64  
 1   Facility Id                   1192267 non-null  int64  
 2   APR DRG Code                  1192267 non-null  int64  
 3   APR Severity of Illness Code  1192267 non-null  int64  
 4   APR Medical Surgical Code     1192267 non-null  object 
 5   Discharges                    1192267 non-null  float64
 6   Mean Charge                   1192267 non-null  float64
 7   Median Charge                 1192267 non-null  float64
 8   Mean Cost                     1192267 non-null  float64
 9   Median Cost                   1192267 non-null  float64
dtypes: float64(5), int64(4), object(1)
memory usage: 100.1+ MB


Now, all the columns are in expected data type of no NA values.

In [28]:
df.describe()

,Year,Facility Id,APR DRG Code,APR Severity of Illness Code,Discharges,Mean Charge,Median Charge,Mean Cost,Median Cost
count,1.192267e+06,1.192267e+06,1.192267e+06,1.192267e+06,1.192267e+06,1.192267e+06,1.192267e+06,1.192267e+06,1.192267e+06
mean,2.013632e+03,9.429071e+02,3.774088e+02,2.342136e+00,2.043550e+01,5.358764e+04,4.911830e+04,1.938937e+04,1.762792e+04
std,3.414268e+00,6.622693e+02,2.473350e+02,1.036409e+00,8.122090e+01,9.371665e+04,8.881965e+04,3.493325e+04,3.300617e+04
min,2.009000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,1.000000e-02,0.000000e+00,0.000000e+00
25%,2.011000e+03,5.130000e+02,1.920000e+02,1.000000e+00,2.000000e+00,1.487500e+04,1.346656e+04,5.855305e+03,5.219980e+03
50%,2.013000e+03,8.950000e+02,3.140000e+02,2.000000e+00,5.000000e+00,2.859500e+04,2.576186e+04,1.054900e+04,9.358980e+03
75%,2.016000e+03,1.306000e+03,5.600000e+02,3.000000e+00,1.500000e+01,5.746721e+04,5.195218e+04,2.076885e+04,1.861554e+04
max,2.021000e+03,1.035500e+04,9.520000e+02,4.000000e+00,6.938000e+03,7.456979e+06,7.456979e+06,7.683086e+06,7.683086e+06


There's no obvious incorrect values from statistic description table.

### Export Data

In [32]:
df.to_csv('cleaned_hospital.csv', index=False)
